# How to store transactions for Anthropic chat in PromptSail

This guide illustrates how to store transactions for Anthropic Claude models using the [Langchain Python SDK](https://python.langchain.com/docs/integrations/chat/anthropic/). It directly interfaces with the Anthropic API Claude 3 family models.

To begin, you'll need an account on [Anthropic](https://anthropic.com/). If you don't have one, create it. 

Next, obtain your API key 🔑 from the [Anthropic console](https://console.anthropic.com/settings/keys).

Then, paste this API key into the `.env` file located in the `examples` folder and assign it to the `ANTHROPIC_API_KEY` variable.

In [1]:
from langchain_anthropic import ChatAnthropic
from dotenv import dotenv_values


config = dotenv_values(".env")

anthropic_key = config["ANTHROPIC_API_KEY"]
print(
    f"Anthropic api key={anthropic_key[0:13]}...{anthropic_key[-3:]}"
)

Anthropic api key=sk-ant-api03-...gAA


Models comparsion you can find at [Anthropic documentation](https://docs.anthropic.com/claude/docs/models-overview#model-comparison).

## This part is just about testing if the API key is correct and you can connect straight to the Anthropic API

In [2]:
from langchain_core.prompts import ChatPromptTemplate

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

In [3]:
chat = ChatAnthropic(temperature=0, anthropic_api_key=anthropic_key, model_name="claude-3-opus-20240229")

In [4]:
chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='저는 파이썬을 사랑합니다.\n\nTranslation:\nI love Python.', response_metadata={'id': 'msg_01VUEHtbpTJGbBVN7h5FGv5i', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 28}}, id='run-c1e509d7-8363-480d-9d3d-2a7f84d6c95f-0')

## Create a request to the Anthropic API via PromptSail proxy

[Run the PromptSail docker images](https://promptsail.com/docs/quick-start-guide/#pull-and-run-the-docker-images-from-ghcr) and go to UI at http://localhost/.
We will have to setup a project and add ai provider. 


Create new project with you `project_slug`or edit existing one for purpose of this example we will use `project1`.

Add your own Anthropic provider by editing the project settings and click "Add AI Provider" button, this will create the mapping between the Anthropic endpoint to promptsail proxy URL. 

In the form set the `api base url` to your Anthropic endpoint to: **'https://api.anthropic.com'**
 
and add meaningfull `deployment name`.

Save the changes by clicking "Add" and "Update" button.
 
In mongo it will create new entry in `ai_providers` array, similar to this one:

```bash
{
     ai_providers: [
        {
            deployment_name: 'anth claude'
            slug: 'anth-claude',
            api_base: 'https://api.anthropic.com',
            description: '',
            provider_name: 'Anthropic'
        }
    ],
}
```

In this case we will use the default `project 1` settings:
* with project_slug -> 'project1' 
* slugified deployment_name -> 'private-anthropic-deployment'

resulting in promptsail proxy url like this: 

**http://localhost:8000/project1/anth-claude**

In [9]:
ps_chat = ChatAnthropic(
    temperature=0, 
    anthropic_api_key=anthropic_key, model_name="claude-3-opus-20240229", anthropic_api_url="http://localhost:8000/project1/anth-claude/"
)

In [10]:
ps_chain = prompt | ps_chat
ps_chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='저는 파이썬을 사랑합니다.\n\nTranslation:\nI love Python.', response_metadata={'id': 'msg_01BPPcr6umzhkCwNKrBKfmwk', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 28}}, id='run-1a4cf04d-b135-41cb-b3bd-a1411dab64a3-0')